- Given a lat lon
- Assume maximum temperature
- convert lat lon to correct projection
- join all times together
- from megacube extract that geospatial point
    - investgate interopolation vs. find nearest
- collapse time by mean
- return float

In [16]:
# GET /helloworld

print('hello world')


hello world


In [17]:
import datetime
import iris

iris.FUTURE.netcdf_promote = True
cubes = iris.load('/opt/data/*.nc')

full_cube = iris.cube.CubeList(cubes).concatenate_cube()
print(full_cube)

ImportError: No module named 'iris'

In [18]:
import cartopy

def transform_coords(cube, lon, lat):
    expected_proj = cube.coords('projection_y_coordinate')[0].coord_system.as_cartopy_crs()
    given_proj = cartopy.crs.PlateCarree()
    return expected_proj.transform_point(lon, lat, given_proj)

ImportError: No module named 'cartopy'

In [19]:
def get_coords(cube, lat, lon):
    x, y = transform_coords(full_cube, lon, lat)
    first_year = next(cube.slices_over('time'))
    samples = [('projection_y_coordinate', y), ('projection_x_coordinate', x)]
    first_point = first_year.interpolate(samples, iris.analysis.Nearest())
    exact_x = first_point.coord('projection_x_coordinate')[0].points[0]
    exact_y = first_point.coord('projection_y_coordinate')[0].points[0]
    
    return(exact_x, exact_y)

In [20]:
def collapse_latlon(cube, x, y):
    x_const = iris.Constraint(projection_x_coordinate=x)
    y_const = iris.Constraint(projection_y_coordinate=y)
    return cube.extract(x_const).extract(y_const)

In [21]:
def extract_date(cube, year=2017, month=2, day=27, past_years=10):
    years = [year - i for i in range(past_years)]
    time_units = cube.coord('time').units
    vals = [time_units.date2num(datetime.datetime(y, month, day)) for y in years]
    time_constraint = iris.Constraint(time=vals)
    return cube.extract(time_constraint)

In [22]:
def mean(cube):
    return cube.collapsed('time', iris.analysis.MEAN)

In [23]:
# GET /temperature/mean
import json

req = json.loads(REQUEST)
args = req['args']
lat = float(args['lat'][0])
lon = float(args['lon'][0])

x, y = get_coords(full_cube, lat, lon)
time_cube = collapse_latlon(full_cube, x, y)
collapsed_time_cube = extract_date(time_cube)
print(mean(collapsed_time_cube).data)

NameError: name 'REQUEST' is not defined